In [5]:
import numpy as np
import plotly.graph_objects as go

In [14]:
# lucro por tonelada de açúcar
c1 = 220.0        
# lucro por 1000L etanol
c2 = 50.0         
# toneladas de cana por 1 tonelada de açúcar
a11 = 8.0         
# toneladas de cana por 1000L de etanol
a12 = 2.0         
# hectares
s = 100.0         
# t cana / hectare
k = 80.0
# acucar minimo do contrato
m1 = 5
# etanol minimo do contrato 
m2 = 200

In [15]:
def geometric_solution(s, k, c1, c2, a11, a12, comodities_intervals=200, profits_lines_qnt=4):
    # maximo produzivel de cana
    t = s * k  

    # maximo produziveis por comodite, consequentemente, pontos interceptos
    sugar_max = t / a11
    etanol_max = t / a12

    # intervalos de producoes das comodities distribuidas num range de 0 ao maximo daquela comoditie, em passos de 'comodities_distribuion'
    # intervalos do maximo de acucar
    sugar_intervals_productions = np.linspace(0, sugar_max, comodities_intervals)
    # intervalos do maximo de etanol removendo o quanto foi produzido de acucar
    etanol_intervals_productions = (t - (a11 * sugar_intervals_productions)) / a12
    # intervalos do maximo de etanol invertido, para que cada valor seja o complemento correspondente ao produzido de acucar
    etanol_intervals_productions_complement = np.clip(etanol_intervals_productions, 0, None)

    # lucros maximos de acucar e etanol
    sugar_profit_max = c1 * sugar_max
    etanol_profit_max = c2 * etanol_max

    # taxa de lucro por comodite, verifica qual comodite é mais rentavel ou se sao igualmente rentaveis
    sugar_profit_rate = c1 / a11
    etanol_profit_rate = c2 / a12
    most_profitable = 0
    if sugar_profit_rate > etanol_profit_rate:
        most_profitable = 1
        better_option_text = f"Ótimo: só açúcar. açúcar={sugar_max:.2f}T, etanol=0L, lucros maximos açúcar={sugar_profit_max:.2f} >= etanol={etanol_profit_max:.2f}"
    elif sugar_profit_rate < etanol_profit_rate:
        most_profitable = 2
        better_option_text = f"Ótimo: só etanol. açúcar=0T, etanol={etanol_max:.2f}L, lucros maximos açúcar={sugar_profit_max:.2f} <= etanol={etanol_profit_max:.2f}"
    else:
        most_profitable = 3
        better_option_text = f"Múltiplas soluções ótimas (qualquer ponto da fronteira). lucros maximos açúcar={sugar_profit_max:.2f} == etanol={etanol_profit_max:.2f}"

    fig = go.Figure()

    # demarcando a fronteira
    fig.add_trace(go.Scatter(
        x=sugar_intervals_productions,
        y=etanol_intervals_productions_complement,
        # mode='lines',
        name="Fronteira"
    ))

    # demarcando a regiao factivel no grafico, area abaixo da fronteira
    fig.add_trace(go.Scatter(
        x=[0, sugar_max, 0, 0],
        y=[0, 0, etanol_max, 0],
        fill='toself',
        fillcolor='rgba(0,100,200,0.08)',
        line=dict(color='rgba(0,0,0,0.2)'),
        name='Região factível'
    ))

    # adicionando 'profits_lines_qnt' linhas de nivel ao grafico
    profit_intervals = np.linspace(0, max(sugar_profit_max,etanol_profit_max), profits_lines_qnt+1)
    profit_intervals = profit_intervals[1:]
    profit_sugar_etanol_intervals = []
    for intervals in profit_intervals:
        sugar_interval_profit_line = np.linspace(0, sugar_max, comodities_intervals)
        etanol_interval_profit_line = (intervals - c1 * sugar_interval_profit_line) / c2
        etanol_interval_profit_line_visible = np.where(etanol_interval_profit_line >= 0, etanol_interval_profit_line, np.nan)
        profit_sugar_etanol_intervals.append((sugar_interval_profit_line, etanol_interval_profit_line_visible))

    for i, (sugar, etanol) in enumerate(profit_sugar_etanol_intervals):
        fig.add_trace(go.Scatter(
            x=sugar,
            y=etanol,
            mode='lines',
            line=dict(dash='dot'),
            name=f"linha {i+1}"
        ))

    # adicionando labels finais ao grafico
    fig.update_layout(
        title="Região factível e linhas de nível da função objetivo",
        xaxis_title="ton açúcar",
        yaxis_title="1000L etanol",
        template='plotly_white',
        annotations=[
            dict(
                x=0.5,
                y=1.12,
                xref="paper",
                yref="paper",
                text=better_option_text,
                showarrow=False,
                align='center',
                font=dict(size=14)
            )
        ]
    )

    return (fig, most_profitable)


In [16]:
(fig, most_profitable)  = geometric_solution(s, k, c1, c2, a11, a12)
fig

In [ ]:
def geometric_solution_minimal_values(
    s, k, c1, c2, a11, a12,
    comodities_intervals=200,
    profits_lines_qnt=4,
    sugar_min_contract=0,
    etanol_min_contract=0
):
    # maximo produzivel de cana
    t = s * k  

    # maximos sem contrato
    sugar_max = t / a11
    etanol_max = t / a12

    fig = go.Figure()

    # ================================
    # (1) Construção da fronteira original
    # ================================
    sugar_intervals_productions = np.linspace(0, sugar_max, comodities_intervals)
    etanol_intervals_productions = (t - (a11 * sugar_intervals_productions)) / a12
    etanol_intervals_productions_complement = np.clip(etanol_intervals_productions, 0, None)

    # ================================
    # (2) Adicionando restrições contratuais
    # ================================
    # As quantidades mínimas obrigatórias
    x1_min = sugar_min_contract
    x2_min = etanol_min_contract

    # checando inviabilidade
    infeasible_contract = False
    if x1_min * a11 + x2_min * a12 > t:
        infeasible_contract = True

    # ================================
    # (3) Construindo fronteira pós-contrato (somente valores >= mínimos)
    # ================================
    sugar_feasible = sugar_intervals_productions[sugar_intervals_productions >= x1_min]
    etanol_feasible = etanol_intervals_productions_complement[sugar_intervals_productions >= x1_min]
    etanol_feasible = np.where(etanol_feasible >= x2_min, etanol_feasible, np.nan)

    # ================================
    # (4) graficar fronteira
    # ================================
    fig.add_trace(go.Scatter(
        x=sugar_intervals_productions,
        y=etanol_intervals_productions_complement,
        name="Fronteira"
    ))

    # ================================
    # (5) região factível com CONTRATOS
    # ================================
    if not infeasible_contract:
        fig.add_trace(go.Scatter(
            x=[x1_min, sugar_max, x1_min, x1_min],
            y=[x2_min, x2_min, etanol_max, x2_min],
            fill='toself',
            fillcolor='rgba(0,100,200,0.15)',
            line=dict(color='rgba(0,0,0,0.3)'),
            name='Região factível (contratos)'
        ))
    else:
        fig.add_annotation(
            x=0.5, y=0.5, xref="paper", yref="paper",
            text="CONTRATOS INVÁLIOS<br>Não há solução factível",
            showarrow=False,
            font=dict(size=20, color="red")
        )

    # ================================
    # (6) Linhas de nível (inalteradas)
    # ================================
    sugar_profit_max = c1 * sugar_max
    etanol_profit_max = c2 * etanol_max

    profit_intervals = np.linspace(0, max(sugar_profit_max, etanol_profit_max), profits_lines_qnt+1)
    profit_intervals = profit_intervals[1:]
    profit_sugar_etanol_intervals = []
    for intervals in profit_intervals:
        sugar_interval_profit_line = np.linspace(0, sugar_max, comodities_intervals)
        etanol_interval_profit_line = (intervals - c1 * sugar_interval_profit_line) / c2
        etanol_interval_profit_line_visible = np.where(etanol_interval_profit_line >= 0, etanol_interval_profit_line, np.nan)
        profit_sugar_etanol_intervals.append((sugar_interval_profit_line, etanol_interval_profit_line_visible))

    for i, (sugar, etanol) in enumerate(profit_sugar_etanol_intervals):
        fig.add_trace(go.Scatter(
            x=sugar,
            y=etanol,
            mode='lines',
            line=dict(dash='dot'),
            name=f"linha {i+1}"
        ))

    # ================================
    # (7) Determinar solução ótima com CONTRATOS
    # ================================
    if infeasible_contract:
        better_option_text = "Sem solução: contratos inviáveis."
        most_profitable = -1
    else:
        # agora a solução ótima vira o ponto da fronteira >= contratos
        sugar_range = sugar_feasible
        etanol_range = etanol_feasible

        # lucro em cada ponto factível
        Z = c1 * sugar_range + c2 * etanol_range

        # eliminar nans
        mask = ~np.isnan(Z)
        if mask.sum() == 0:
            better_option_text = "Sem solução: região factível vazia após contratos."
            most_profitable = -1
        else:
            idx = np.argmax(Z[mask])
            best_sugar = sugar_range[mask][idx]
            best_etanol = etanol_range[mask][idx]
            best_profit = Z[mask][idx]

            better_option_text = (
                f"Ótimo com contratos: açúcar={best_sugar:.2f}, "
                f"etanol={best_etanol:.2f}, lucro={best_profit:.2f}"
            )
            most_profitable = 10  # novo código para identificar "ótimo com contratos"

            # destacar solução
            fig.add_trace(go.Scatter(
                x=[best_sugar],
                y=[best_etanol],
                mode="markers",
                marker=dict(size=12, color="red"),
                name="Solução ótima"
            ))

    # Anotação superior
    fig.update_layout(
        title="Região factível com contratos e linhas de nível",
        xaxis_title="ton açúcar",
        yaxis_title="1000L etanol",
        template='plotly_white',
        annotations=[
            dict(
                x=0.5, y=1.10, xref="paper", yref="paper",
                text=better_option_text,
                showarrow=False,
                font=dict(size=14)
            )
        ]
    )

    return fig, most_profitable


In [18]:
(fig, most_profitable)  = geometric_solution_minimal_values(s, k, c1, c2, a11, a12, sugar_min_contract=m1, etanol_min_contract=m2)
fig